## Tutorial for all embedding used in this notebook
https://www.analyticsvidhya.com/blog/2020/08/top-4-sentence-embedding-techniques-using-python/

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [38]:
import pandas as pd
import re
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import re
import string
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm.auto import tqdm
!pip install demoji
import demoji
demoji.download_codes()
nltk.download('stopwords') 
nltk.download('wordnet')

... OK (Got response in 0.15 seconds)
Writing emoji data to /root/.demoji/codes.json ...
... OK
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [39]:
df = pd.read_json('/content/drive/MyDrive/sarcasm detection/Datasets/twitter/sarcasm_detection_shared_task_twitter_training.jsonl',lines=True)
df1 = pd.read_json('/content/drive/MyDrive/sarcasm detection/Datasets/twitter/sarcasm_detection_shared_task_twitter_testing.jsonl',lines=True)
#df1['response'] = df1['response'].apply(preprocess_text)

In [40]:
df.head()

,label,response,context
0,SARCASM,@USER @USER @USER I don't get this .. obviousl...,[A minor child deserves privacy and should be ...
1,SARCASM,@USER @USER trying to protest about . Talking ...,[@USER @USER Why is he a loser ? He's just a P...
2,SARCASM,@USER @USER @USER He makes an insane about of ...,[Donald J . Trump is guilty as charged . The e...
3,SARCASM,@USER @USER Meanwhile Trump won't even release...,[Jamie Raskin tanked Doug Collins . Collins lo...
4,SARCASM,@USER @USER Pretty Sure the Anti-Lincoln Crowd...,[Man ... y ’ all gone “ both sides ” the apoca...


In [41]:
df.iloc[57,:]['context']

['@USER sweet ! I always wanted to explore this country !',
 '@USER You sure ? I was denied entry two years ago for " security reasons " . @USER']

In [ ]:
for i in range(50,100):
  if df.loc[i,'label']=='SARCASM':
    print(i,df.loc[i,'response'])

In [43]:
print(df['context'][11])
print(df['response'][11])
print(df['label'][11])

['. @USER : " Democrats have become radicalized by this last election . " They\'ve concluded Clinton was too moderate and they need more Bernies .', '. @USER predicts Dems will now become " obstructionists at a level we\'ve never seen . " They\'ll " filibuster absolutely everything they can . "']
@USER @USER @USER Much like Trump , Sen Cruz Tragically born without an bone . _
SARCASM


In [44]:
df['context_resp']=df['label']
for i in range(5000):
  df['context_resp'][i]=''.join(df['context'][i]) + df['response'][i]

print(df['context'][12])
print(df['response'][12])
print(df['context_resp'][12])

['Matt here . Rarely comment on the president , but having someone who is not Jewish tell Jews what they should or shouldn ’ t think is beyond the pale and it should not go unchallenged . <URL>', '@USER It seems there is a long history of non-Jews telling Jews what to think .']
@USER @USER Hey , but what do they have to lose ? Asking for some black friends .
Matt here . Rarely comment on the president , but having someone who is not Jewish tell Jews what they should or shouldn ’ t think is beyond the pale and it should not go unchallenged . <URL>@USER It seems there is a long history of non-Jews telling Jews what to think .@USER @USER Hey , but what do they have to lose ? Asking for some black friends .


In [45]:
def preprocess_text(text):
    # Tokenise words while ignoring punctuation(https://www.nltk.org/_modules/nltk/tokenize/regexp.html)
    text = re.sub(r"@USER","", text)
    text = re.sub(r'<URL>','',text)
    text = demoji.replace_with_desc(text)
    text = re.sub(r':','',text)
    # tokeniser = RegexpTokenizer(r'\s+', gaps=True)
    # tokeniser = RegexpTokenizer('\w+|\$[\d\.]+|\S+')
    tokeniser = RegexpTokenizer(r'\w+')
    tokens = tokeniser.tokenize(text)

    # Lowercase and lemmatise 
    lemmatiser = WordNetLemmatizer()
    lemmas = [lemmatiser.lemmatize(token.lower(), pos='v') for token in tokens]
    
    # Remove stopwords
    keywords= [lemma for lemma in lemmas if lemma not in stopwords.words('english')]
    
    # stemming
    ps=PorterStemmer()
    keywords=[ps.stem(x) for x in keywords]
    return keywords

def preprocess_text2(tweet):
    """Process tweet function.
    Input:
        tweet: a string containing a tweet
    Output:
        tweets_clean: a list of words containing the processed tweet

    """
    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')
    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
    # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    # tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)

    tweets_clean = []
    for word in tweet_tokens:
        if (word not in stopwords_english and  # remove stopwords
                word not in string.punctuation):  # remove punctuation (optional)
            # tweets_clean.append(word)
            stem_word = stemmer.stem(word)  # stemming word
            tweets_clean.append(stem_word)

    return tweets_clean

In [46]:
for i in range(5):
  print(df['response'][i])
  print(preprocess_text(df['response'][i]))
  print(preprocess_text2(df['response'][i]))
  print()
  print(df['context_resp'][i])
  print(preprocess_text(df['context_resp'][i]))
  print(preprocess_text2(df['context_resp'][i]))
  print()

@USER @USER @USER I don't get this .. obviously you do care or you would've moved right along .. instead you decided to care and troll her ..
['get', 'obvious', 'care', 'would', 'move', 'right', 'along', 'instead', 'decid', 'care', 'troll']
['get', '..', 'obvious', 'care', "would'v", 'move', 'right', 'along', '..', 'instead', 'decid', 'care', 'troll', '..']

A minor child deserves privacy and should be kept out of politics . Pamela Karlan , you should be ashamed of your very angry and obviously biased public pandering , and using a child to do it .@USER If your child isn't named Barron ... #BeBest Melania couldn't care less . Fact . 💯@USER @USER @USER I don't get this .. obviously you do care or you would've moved right along .. instead you decided to care and troll her ..
['minor', 'child', 'deserv', 'privaci', 'keep', 'polit', 'pamela', 'karlan', 'asham', 'angri', 'obvious', 'bia', 'public', 'pander', 'use', 'child', 'child', 'name', 'barron', 'bebest', 'melania', 'care', 'less', 'fa

In [47]:
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, GridSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, accuracy_score

In [48]:
print(df['label'].value_counts())
print(df1['label'].value_counts())

SARCASM        2500
NOT_SARCASM    2500
Name: label, dtype: int64
NOT_SARCASM    900
SARCASM        900
Name: label, dtype: int64


In [49]:
from sklearn.utils import shuffle
X_train, Y_train = shuffle(df['response'],df['label'],random_state=123)
X_test, Y_test = df1['response'],df1['label']

#**TF-idf**

In [50]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [51]:
vectoriser = TfidfVectorizer(analyzer=preprocess_text)
# Fit to the data and transform to feature matrix
X_train_tfidf = vectoriser.fit_transform(X_train)
print(X_train_tfidf.shape)

X_test_tfidf = vectoriser.transform(X_test)
print(X_test_tfidf.shape)

(5000, 9496)
(1800, 9496)


In [52]:
#all ml models
# !pip install nbsvm-sklearn
# from nbsvm import NBSVMClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier

# first, initialize the classificators
# nbsvm = NBSVMClassifier()
tree= DecisionTreeClassifier(random_state=24) # using the random state for reproducibility
forest= RandomForestClassifier(random_state=24)
knn= KNeighborsClassifier()
svm= SVC(random_state=24)
xboost= XGBClassifier(random_state=24)


# now, create a list with the objects 
models= [tree, forest, knn, svm, xboost]
#models=[nbsvm]
for model in models:
    model.fit(X_train_tfidf, Y_train) # fit the model
    y_pred= model.predict(X_test_tfidf) # then predict on the test set
    accuracy= accuracy_score(Y_test, y_pred) # this gives us how often the algorithm predicted correctly
    clf_report= classification_report(Y_test, y_pred) # with the report, we have a bigger picture, with precision and recall for each class
    print(f"The accuracy of model {type(model).__name__} is {accuracy:.2f}")
    print(clf_report)
    print("\n")

The accuracy of model DecisionTreeClassifier is 0.58
              precision    recall  f1-score   support

 NOT_SARCASM       0.59      0.56      0.57       900
     SARCASM       0.58      0.60      0.59       900

    accuracy                           0.58      1800
   macro avg       0.58      0.58      0.58      1800
weighted avg       0.58      0.58      0.58      1800



The accuracy of model RandomForestClassifier is 0.63
              precision    recall  f1-score   support

 NOT_SARCASM       0.66      0.54      0.59       900
     SARCASM       0.61      0.71      0.66       900

    accuracy                           0.63      1800
   macro avg       0.63      0.63      0.63      1800
weighted avg       0.63      0.63      0.63      1800



The accuracy of model KNeighborsClassifier is 0.59
              precision    recall  f1-score   support

 NOT_SARCASM       0.60      0.56      0.58       900
     SARCASM       0.59      0.63      0.61       900

    accuracy         

#**Doc2Vec**

In [53]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [54]:
tagged_data = [TaggedDocument(words=preprocess_text(_d), tags=[str(i)]) for i, _d in enumerate(X_train)]
tagged_data[:5]

[TaggedDocument(words=['khanna', 'huge', 'fan', 'humbl', 'soul', 'wonder', 'human', 'chef', 'thank', 'appreci', 'asim', 'fold', 'hand', 'truli', 'nice', 'human', 'insid', 'love', 'asim', 'request', 'pleas', 'convey', 'show', 'trophi'], tags=['0']),
 TaggedDocument(words=['two', 'quot', 'cenk', 'reason', 'would', 'equal', 'women', 'incred', 'weak', 'man', 'women', 'equal', 'angri', 'sure', 'sound', 'like', 'misogynist'], tags=['1']),
 TaggedDocument(words=['happi', 'equal', 'valu', 'state', 'want', 'happi', 'matter', 'prefer', 'would', 'rather', 'laugh', 'cri', 'yell', 'act', 'cold', 'robot', 'laugh', 'smile', 'enjoy', 'time', 'earth', 'globe', 'show', 'america', 'parti', 'face', 'happi', 'enjoy', 'laughter'], tags=['2']),
 TaggedDocument(words=['make', 'sens', 'shock', 'proverbi', 'goalpost', 'right', 'alway', 'firmli', 'ground', 'realiti', 'problem', 'read', 'comprehens', 'concern', 'simpleton', 'part', 'literaci', 'criticalthink'], tags=['3']),
 TaggedDocument(words=['trust', 'mafia'

In [55]:
max_epochs =100
vec_size = 20
alpha = 0.025

model = Doc2Vec(vector_size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1)
  
model.build_vocab(tagged_data)


for epoch in tqdm(range(max_epochs)):
    # print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=1)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    # model.min_alpha = model.alpha

model.save("d2v.model")
print("Model Saved")


Model Saved


In [56]:
model= Doc2Vec.load("d2v.model")

X_train_d2v=[model.infer_vector(preprocess_text(X_train[i])) for i in range(len(X_train))]
X_train_d2v=np.array(X_train_d2v)

X_test_d2v=[model.infer_vector(preprocess_text(X_test[i])) for i in range(len(X_test))]
X_test_d2v=np.array(X_test_d2v)

In [57]:
# X_train1, X_test1, Y_train1, Y_test1=train_test_split(X_train_d2v,Y_train,random_state=42)
X_train1, X_test1, Y_train1, Y_test1=X_train_d2v, X_test_d2v, Y_train, Y_test

In [58]:
#all ml models
# !pip install nbsvm-sklearn
# from nbsvm import NBSVMClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier

# first, initialize the classificators
# nbsvm = NBSVMClassifier()
tree= DecisionTreeClassifier(random_state=24) # using the random state for reproducibility
forest= RandomForestClassifier(random_state=24)
knn= KNeighborsClassifier()
svm= SVC(random_state=24)
xboost= XGBClassifier(random_state=24)

# X_test_tfidf = vectoriser.transform(X_test)
# print(X_test_tfidf.shape)
# now, create a list with the objects 
models= [tree, forest, knn, svm, xboost]
#models=[nbsvm]
for model in models:
    model.fit(X_train1, Y_train1) # fit the model
    y_pred= model.predict(X_test1) # then predict on the test set
    accuracy= accuracy_score(Y_test1, y_pred) # this gives us how often the algorithm predicted correctly
    clf_report= classification_report(Y_test1, y_pred) # with the report, we have a bigger picture, with precision and recall for each class
    print(f"The accuracy of model {type(model).__name__} is {accuracy:.2f}")
    print(clf_report)
    print("\n")

The accuracy of model DecisionTreeClassifier is 0.51
              precision    recall  f1-score   support

 NOT_SARCASM       0.51      0.56      0.53       900
     SARCASM       0.51      0.47      0.49       900

    accuracy                           0.51      1800
   macro avg       0.51      0.51      0.51      1800
weighted avg       0.51      0.51      0.51      1800



The accuracy of model RandomForestClassifier is 0.49
              precision    recall  f1-score   support

 NOT_SARCASM       0.49      0.49      0.49       900
     SARCASM       0.49      0.50      0.50       900

    accuracy                           0.49      1800
   macro avg       0.49      0.49      0.49      1800
weighted avg       0.49      0.49      0.49      1800



The accuracy of model KNeighborsClassifier is 0.51
              precision    recall  f1-score   support

 NOT_SARCASM       0.51      0.54      0.53       900
     SARCASM       0.51      0.48      0.50       900

    accuracy         

In [59]:
# We manually tune the parameters(alpha,vec_size,max_size) but unfortunately maximum accuracy reached till 0.52. 



#**Universal Sentence Encoder**

In [ ]:
!pip3 install --upgrade tensorflow-gpu
# Install TF-Hub.
!pip3 install tensorflow-hub

In [61]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np

In [62]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" 
model = hub.load(module_url)
print ("module %s loaded" % module_url)

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [63]:
X_train_use = model(list(X_train.values))
X_test_use  = model(list(X_test.values))
print(X_train_use.shape)
print(X_test_use.shape)

(5000, 512)
(1800, 512)


In [64]:
X_train3, X_test3, Y_train3, Y_test3=X_train_use, X_test_use, Y_train, Y_test

In [65]:
#all ml models
# !pip install nbsvm-sklearn
# from nbsvm import NBSVMClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB

# first, initialize the classificators
# nbsvm = NBSVMClassifier()
tree= DecisionTreeClassifier(random_state=24) # using the random state for reproducibility
forest= RandomForestClassifier(random_state=24)
knn= KNeighborsClassifier()
svm= SVC(random_state=24)
xboost= XGBClassifier(random_state=24)
gnb = GaussianNB()

# now, create a list with the objects 
models= [tree, forest, knn, svm, xboost, gnb]
#models=[nbsvm]
for model in models:
    model.fit(X_train3, Y_train3) # fit the model
    y_pred= model.predict(np.array(X_test3)) # then predict on the test set
    accuracy= accuracy_score(Y_test3, y_pred) # this gives us how often the algorithm predicted correctly
    clf_report= classification_report(Y_test3, y_pred) # with the report, we have a bigger picture, with precision and recall for each class
    print(f"The accuracy of model {type(model).__name__} is {accuracy:.2f}")
    print(clf_report)
    print("\n")

The accuracy of model DecisionTreeClassifier is 0.59
              precision    recall  f1-score   support

 NOT_SARCASM       0.60      0.57      0.58       900
     SARCASM       0.59      0.61      0.60       900

    accuracy                           0.59      1800
   macro avg       0.59      0.59      0.59      1800
weighted avg       0.59      0.59      0.59      1800



The accuracy of model RandomForestClassifier is 0.67
              precision    recall  f1-score   support

 NOT_SARCASM       0.69      0.62      0.66       900
     SARCASM       0.66      0.72      0.69       900

    accuracy                           0.67      1800
   macro avg       0.67      0.67      0.67      1800
weighted avg       0.67      0.67      0.67      1800



The accuracy of model KNeighborsClassifier is 0.65
              precision    recall  f1-score   support

 NOT_SARCASM       0.69      0.55      0.61       900
     SARCASM       0.63      0.76      0.69       900

    accuracy         

#**Using Sentence Transformer embedding**

https://github.com/UKPLab/sentence-transformers

In [66]:
pip install -U sentence-transformers

     |████████████████████████████████| 92kB 2.3MB/s 
     |████████████████████████████████| 2.5MB 9.2MB/s 
     |████████████████████████████████| 1.2MB 30.1MB/s 
     |████████████████████████████████| 3.3MB 20.7MB/s 
     |████████████████████████████████| 901kB 37.4MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.0.0-cp37-none-any.whl size=126711 sha256=837ab8799209d39cfcb0a0b5744bdd200d6a3632117fa0c10abc5eab3323bbcb
  Stored in directory: /root/.cache/pip/wheels/38/d2/98/d191289a877a34c68aa67e05179521e060f96394a3e9336be6
Successfully built sentence-transformers
ERROR: transformers 4.8.2 has requirement huggingface-hub==0.0.12, but you'll have huggingface-hub 0.0.13 which is incompatible.


In [67]:
from sentence_transformers import SentenceTransformer
# sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')
# sbert_model = SentenceTransformer('paraphrase-distilroberta-base-v1')
sbert_model = SentenceTransformer('stsb-mpnet-base-v2')

In [68]:
# sgd_clf = SGDClassifier(random_state=123)
from tqdm.auto import tqdm
import numpy as np
query = "I had pizza and pasta"
# sentence_embeddings = sbert_model.encode(query) 
sentence_embeddings = [sbert_model.encode(i.lower()) for i in tqdm(X_train)]

Exception ignored in: <generator object tqdm_notebook.__iter__ at 0x7f81740ace50>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/tqdm/notebook.py", line 220, in __iter__
    self.sp(bar_style='danger')
AttributeError: 'tqdm_notebook' object has no attribute 'sp'


KeyboardInterrupt: ignored

In [ ]:
query = "I had pizza and pasta"
# sentence_embeddings = sbert_model.encode(query) 
test_embeddings = [sbert_model.encode(i.lower()) for i in tqdm(X_test)]

In [ ]:
sentence_embeddings=np.array(sentence_embeddings)
test_embeddings=np.array(test_embeddings)

In [ ]:
# X_train2, X_test2, Y_train2, Y_test2=train_test_split(sentence_embeddings,Y_train,random_state=42)
X_train2, X_test2, Y_train2, Y_test2=sentence_embeddings, test_embeddings, Y_train, Y_test

In [ ]:
#all ml models
!pip install nbsvm-sklearn
from nbsvm import NBSVMClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB

# first, initialize the classificators
nbsvm = NBSVMClassifier()
tree= DecisionTreeClassifier(random_state=24) # using the random state for reproducibility
forest= RandomForestClassifier(random_state=24)
knn= KNeighborsClassifier()
svm= SVC(random_state=24, probability=True)
xboost= XGBClassifier(random_state=24)
gnb = GaussianNB()
# y_pred = gnb.fit(X_train, y_train).predict(X_test)

# X_test_tfidf = vectoriser.transform(X_test)
# print(X_test_tfidf.shape)
# now, create a list with the objects 
# models= [tree, forest, knn, svm, xboost, gnb]
models=[svm]
for model in models:
    model.fit(X_train2, Y_train2) # fit the model
    y_pred= model.predict(X_test2) # then predict on the test set
    accuracy= accuracy_score(Y_test2, y_pred) # this gives us how often the algorithm predicted correctly
    clf_report= classification_report(Y_test2, y_pred) # with the report, we have a bigger picture, with precision and recall for each class
    print(f"The accuracy of model {type(model).__name__} is {accuracy:.2f}")
    print(clf_report)
    print("\n")

In [ ]:
svm= SVC(random_state=24, probability=True)
svm.fit(X_train2, Y_train2)
y_pred_prob = svm.predict_proba(X_test2)

In [ ]:
y_pred_prob=pd.DataFrame(y_pred_prob)
y_pred_prob.to_csv('Bert_SVM.csv')

In [ ]:
# without any fine tunning we got maximum accuracy 0.69 


# Using Infercent embedding.



In [69]:
! mkdir encoder
! curl -Lo encoder/infersent2.pkl https://dl.fbaipublicfiles.com/infersent/infersent2.pkl
  
! mkdir GloVe
! curl -Lo GloVe/glove.840B.300d.zip http://nlp.stanford.edu/data/glove.840B.300d.zip
! unzip GloVe/glove.840B.300d.zip -d GloVe/

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  146M  100  146M    0     0  17.6M      0  0:00:08  0:00:08 --:--:-- 20.9M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0   315    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0   352    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 2075M  100 2075M    0     0  5058k      0  0:07:00  0:07:00 --:--:-- 5792k
Archive:  GloVe/glove.840B.300d.zip
  inflating: GloVe/glove.840B.300d.txt  


First download models.py file and save in working directory from https://github.com/facebookresearch/InferSent

In [71]:
from models import InferSent
import torch

V = 2
MODEL_PATH = 'encoder/infersent%s.pkl' % V
params_model = {'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
                'pool_type': 'max', 'dpout_model': 0.0, 'version': V}
modeli = InferSent(params_model)
modeli.load_state_dict(torch.load(MODEL_PATH))

W2V_PATH = '/content/GloVe/glove.840B.300d.txt'
modeli.set_w2v_path(W2V_PATH)

In [73]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [74]:
# sentences = ["I ate dinner.", 
#        "We had a three-course meal.", 
#        "Brad came to dinner with us.",
#        "He loves fish tacos.",
#        "In the end, we all felt like we ate too much.",
#        "We all agreed; it was a magnificent evening."]
modeli.build_vocab(X_train, tokenize=True)

Found 14038(/15792) words with w2v vectors
Vocab size : 14038


In [75]:
# query = "I had pizza and pasta"
train_vec=[]
for i in tqdm(X_train):
  query_vec = modeli.encode(i[0])
  query_vec=query_vec.reshape((4096,))
  train_vec.append(query_vec)

In [76]:
test_vec=[]
for i in tqdm(X_test):
  q_vec = modeli.encode(i[0])
  q_vec= q_vec.reshape((4096,))
  test_vec.append(q_vec)

In [77]:
train_vec=np.array(train_vec)

test_vec=np.array(test_vec)

In [78]:
X_train3, X_test3, Y_train3, Y_test3 = train_vec, test_vec, Y_train, Y_test
X_train3.shape

(5000, 4096)

In [79]:
#all ml models
# !pip install nbsvm-sklearn
from nbsvm import NBSVMClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB

# first, initialize the classificators
nbsvm = NBSVMClassifier()
tree= DecisionTreeClassifier(random_state=24) # using the random state for reproducibility
forest= RandomForestClassifier(random_state=24)
knn= KNeighborsClassifier()
svm= SVC(random_state=24)
xboost= XGBClassifier(random_state=24)
gnb = GaussianNB()
# y_pred = gnb.fit(X_train, y_train).predict(X_test)

# X_test_tfidf = vectoriser.transform(X_test)
# print(X_test_tfidf.shape)
# now, create a list with the objects 
models= [tree, forest, knn, svm, xboost, gnb]
#models=[nbsvm]
for model in models:
    model.fit(X_train3, Y_train3) # fit the model
    y_pred= model.predict(X_test3) # then predict on the test set
    accuracy= accuracy_score(Y_test3, y_pred) # this gives us how often the algorithm predicted correctly
    clf_report= classification_report(Y_test3, y_pred) # with the report, we have a bigger picture, with precision and recall for each class
    print(f"The accuracy of model {type(model).__name__} is {accuracy:.2f}")
    print(clf_report)
    print("\n")

  Created wheel for nbsvm-sklearn: filename=nbsvm_sklearn-0.0.6-cp37-none-any.whl size=3401 sha256=4bc836fc5cb137d0689a28ad85c9aa4405832ebf8336997b0a666f127e579d57
  Stored in directory: /root/.cache/pip/wheels/bb/c2/76/acd1a8577a0f0c3bb9abd8b8d9b64911f3a1748c7a53f011cf
Successfully built nbsvm-sklearn
The accuracy of model DecisionTreeClassifier is 0.49
              precision    recall  f1-score   support

 NOT_SARCASM       0.00      0.00      0.00       900
     SARCASM       0.50      0.99      0.66       900

    accuracy                           0.49      1800
   macro avg       0.25      0.49      0.33      1800
weighted avg       0.25      0.49      0.33      1800



The accuracy of model RandomForestClassifier is 0.49
              precision    recall  f1-score   support

 NOT_SARCASM       0.00      0.00      0.00       900
     SARCASM       0.50      0.99      0.66       900

    accuracy                           0.49      1800
   macro avg       0.25      0.49      0.33

# Using NB-SVM

In [ ]:
import numpy as np
from keras.layers.core import Activation
from keras.models import Model
from keras.layers import Input, Embedding, Flatten, dot
from keras import backend as K
from keras.optimizers import Adam
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.datasets import load_files

In [ ]:

# PATH_TO_IMDB = r'./data/aclImdb'

def load_ata(X_train,Y_train,X_test,Y_test):
    veczr =  CountVectorizer(ngram_range=(1,3), binary=True, 
                             token_pattern=r'\w+', 
                             max_features=800000)
    dtm_train = veczr.fit_transform(X_train)
    dtm_test = veczr.transform(X_test)
    print("document-term matrix shape (training): (%s, %s)" % (dtm_train.shape))
    print("document-term matrix shape (test): (%s, %s)" % (dtm_train.shape))
    num_words = len([v for k,v in veczr.vocabulary_.items()]) + 1 # add 1 for 0 padding
    print('vocab size:%s' % (num_words))
  
    return (dtm_train, dtm_test), (Y_train, Y_test), num_words

(dtm_train, dtm_test), (y_train, y_test), num_words = load_imdb_data(X_train,Y_train,X_test,Y_test)

In [ ]:


def dtm2wid(dtm, maxlen=2000):
    x = []
    nwds = []
    for idx, row in enumerate(dtm):
        seq = []
        indices = (row.indices + 1).astype(np.int64)
        np.append(nwds, len(indices))
        data = (row.data).astype(np.int64)
        count_dict = dict(zip(indices, data))
        for k,v in count_dict.items():
            seq.extend([k]*v)
        num_words = len(seq)
        nwds.append(num_words)
        # pad up to maxlen
        if num_words < maxlen: 
            seq = np.pad(seq, (maxlen - num_words, 0), mode='constant')
        # truncate down to maxlen
        else:                  
            seq = seq[-maxlen:]
        x.append(seq)
    nwds = np.array(nwds)
    print('sequence stats: avg:%s, max:%s, min:%s' % (nwds.mean(), nwds.max(), nwds.min()) )
    return np.array(x)

maxlen = 2000
x_train = dtm2wid(dtm_train, maxlen=maxlen)
x_test = dtm2wid(dtm_test, maxlen=maxlen)

In [ ]:
def pr(dtm, y, y_i):
    p = dtm[y==y_i].sum(0)
    return (p+1) / ((y==y_i).sum()+1)
# nbratios = np.log(pr(dtm_train, y_train, 1)/pr(dtm_train, y_train, 0))
# nbratios = np.squeeze(np.asarray(nbratios))

In [ ]:
def get_model(num_words, maxlen, nbratios=None):
    embedding_matrix = np.zeros((num_words, 1))
    for i in range(1, num_words): # skip 0, the padding value
        if nbratios is not None:
            # if log-count ratios are supplied, then it's NBSVM
            embedding_matrix[i] = nbratios[i-1]
        else:
            # if log-count rations are not supplied, this reduces to a logistic regression
            embedding_matrix[i] = 1

    # set up the model
    inp = Input(shape=(maxlen,))
    r = Embedding(num_words, 1, input_length=maxlen, weights=[embedding_matrix], trainable=False)(inp)
    x = Embedding(num_words, 1, input_length=maxlen, embeddings_initializer='glorot_normal')(inp)
    x = dot([r,x], axes=1)
    x = Flatten()(x)
    x = Activation('sigmoid')(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy',
                  optimizer=Adam(lr=0.0005),
                  metrics=['accuracy'])
    return model

In [ ]:
x_train.shape
Y_train=list(y_train)
ytrain=[int(i=="SARCASM") for i in Y_train]
Y_train=list(y_test)
ytest=[int(i=="SARCASM") for i in Y_test]
ytrain


In [ ]:
model = get_model(num_words, maxlen, nbratios=None)
model.fit(np.array(x_train), np.array(ytrain),
          batch_size=32,
          epochs=5,
          validation_data=(np.array(x_test),np.array(ytest)))

# Bag of Words

In [10]:
import pandas as pd
import re
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import re
import string
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
nltk.download('stopwords') 
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
df = pd.read_json('/content/drive/My Drive/sarcasm/twitter/sarcasm_detection_shared_task_twitter_training.jsonl',lines=True)
df1 = pd.read_json('/content/drive/My Drive/sarcasm/twitter/sarcasm_detection_shared_task_twitter_testing.jsonl',lines=True)
#df1['response'] = df1['response'].apply(preprocess_text)

In [5]:
def preprocess_text(text):
    # Tokenise words while ignoring punctuation
    text = re.sub(r"@USER","", text)
    tokeniser = RegexpTokenizer(r'\w+')
    tokens = tokeniser.tokenize(text)
    
    # Lowercase and lemmatise 
    lemmatiser = WordNetLemmatizer()
    lemmas = [lemmatiser.lemmatize(token.lower(), pos='v') for token in tokens]
    
    # Remove stopwords
    keywords= [lemma for lemma in lemmas if lemma not in stopwords.words('english')]
    return keywords

def preprocess_text2(tweet):
    """Process tweet function.
    Input:
        tweet: a string containing a tweet
    Output:
        tweets_clean: a list of words containing the processed tweet

    """
    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')
    # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    # tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)

    tweets_clean = []
    for word in tweet_tokens:
        if (word not in stopwords_english and  # remove stopwords
                word not in string.punctuation):  # remove punctuation
            # tweets_clean.append(word)
            stem_word = stemmer.stem(word)  # stemming word
            tweets_clean.append(stem_word)

    return tweets_clean
import re


def clean_text(text):
    """
    Applies some pre-processing on the given text.

    Steps :
    - Removing HTML tags
    - Removing punctuation
    - Lowering text
    """
    text = re.sub(r"@USER","", text)
    # remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    
    # remove the characters [\], ['] and ["]
    text = re.sub(r"\\", "", text)    
    text = re.sub(r"\'", "", text)    
    text = re.sub(r"\"", "", text)    
    
    # convert text to lowercase
    text = text.strip().lower()
    
    # replace punctuation characters with spaces
    filters='!"\'#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'
    translate_dict = dict((c, " ") for c in filters)
    translate_map = str.maketrans(translate_dict)
    text = text.translate(translate_map)

    return text

In [6]:
print(clean_text("this is check oplease !!!! cdcnsdj$$V4"))


this is check oplease      cdcnsdj  v4


In [7]:
from sklearn.utils import shuffle
X_train, Y_train = shuffle(df['response'],df['label'],random_state=13)
X_test, Y_test = df1['response'],df1['label']

In [11]:
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC
vectorizer = CountVectorizer(stop_words="english",
                             preprocessor=clean_text)
training_features = vectorizer.fit_transform(X_train)    
test_features = vectorizer.transform(X_test)

In [12]:
#all ml models
#!pip install nbsvm-sklearn
#from nbsvm import NBSVMClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier

# first, initialize the classificators
#nbsvm = NBSVMClassifier()
tree= DecisionTreeClassifier(random_state=24) # using the random state for reproducibility
forest= RandomForestClassifier(random_state=24)
knn= KNeighborsClassifier()
svm= SVC(random_state=24)
xboost= XGBClassifier(random_state=24) 
models= [tree, forest, knn, svm, xboost]
for model in models:
    model.fit(training_features, Y_train) # fit the model
    y_pred= model.predict(test_features) # then predict on the test set
    accuracy= accuracy_score(Y_test, y_pred) # this gives us how often the algorithm predicted correctly
    clf_report= classification_report(Y_test, y_pred) # with the report, we have a bigger picture, with precision and recall for each class
    print(f"The accuracy of model {type(model).__name__} is {accuracy:.2f}")
    print(clf_report)
    print("\n")

The accuracy of model DecisionTreeClassifier is 0.58
              precision    recall  f1-score   support

 NOT_SARCASM       0.59      0.51      0.55       900
     SARCASM       0.57      0.64      0.60       900

    accuracy                           0.58      1800
   macro avg       0.58      0.58      0.58      1800
weighted avg       0.58      0.58      0.58      1800



The accuracy of model RandomForestClassifier is 0.62
              precision    recall  f1-score   support

 NOT_SARCASM       0.66      0.48      0.56       900
     SARCASM       0.59      0.75      0.66       900

    accuracy                           0.62      1800
   macro avg       0.63      0.62      0.61      1800
weighted avg       0.63      0.62      0.61      1800



The accuracy of model KNeighborsClassifier is 0.52
              precision    recall  f1-score   support

 NOT_SARCASM       0.52      0.65      0.58       900
     SARCASM       0.53      0.39      0.45       900

    accuracy         